In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import tqdm

In [ ]:
section_dict = {
    '기획·전략' : '10026',
    '법무·사무·총무' : '10027',
    '인사·HR' : '10028',
    '회계·세무' : '10029',
    '마케팅·광고·MD' : '10030',
    '개발·데이터' : '10031',
    '디자인' : '10032',
    '물류·무역' : '10033',
    '운전·운송·배송' : '10034',
    '영업' : '10035',
    '고객상담·TM' : '10036',
    '금융·보험' : '10037',
    '식·음료' : '10038',
    '고객서비스·리테일' : '10039',
    '엔지니어링·설계' : '10040',
    '제조·생산' : '10041',
    '교육' : '10042',
    '건축·시설' : '10043',
    '의료·바이오' : '10044',
    '미디어·문화·스포츠' : '10045',
    '공공·복지' : '10046'
}

In [ ]:
df = pd.DataFrame(columns=["section", "company", "date", "work_type", "job", "apply_info", "question", "answer"])
df.to_csv("./data/coverletter_df.csv", encoding="utf-8-sig", index=False)

In [ ]:
for section_name, section_id in section_dict.items():
    
    page = 1
    BASE_URL = "https://www.jobkorea.co.kr/starter/PassAssay" + f"?schPart={section_id}"
    
    print(section_name + " 시작!")

    while True:
        URL = BASE_URL + f"&Page={page}"
        response = requests.get(URL)
        soup = BeautifulSoup(response.text, "html.parser")

        if soup.find(class_="scNoSelect"):
            break

        else:
            essay_list = soup.find(class_="selfLists").find_all(class_="txBx")

            for essay in essay_list:

                # Job Info
                url = "https://www.jobkorea.co.kr/" + essay.find("a")["href"]
                company = essay.find(class_="titTx").text
                season = essay.find(class_="career").text
                work_type = essay.find_all(class_="field")[0].text
                job = essay.find_all(class_="field")[1].text

                # Essay Info

                try:
                    response = requests.get(url)
                    soup = BeautifulSoup(response.text, "html.parser")
                    apply_info = soup.find(class_="specLists").text.strip().split('\n')[:-1]

                    qna_list = soup.find(class_="qnaLists")

                    q_list = []
                    for q in qna_list.find_all("dt"):
                        q_text = q.find(class_="tx").text
                        q_list.append(q_text)

                    a_list = []
                    for a in qna_list.find_all("dd"):
                        a_text = a.find(class_="tx").text
                        a_text = a_text[:-20].replace("\r", "").replace("답변", "").strip()
                        a_list.append(a_text)

                    qna_text = list(zip(q_list, a_list))

                    # Save data
                    insert_data = []
                    for text in qna_text:
                        data = []
                        data.append(section_name)
                        data.append(company)
                        data.append(season)
                        data.append(work_type)
                        data.append(job)
                        data.append(apply_info)
                        data.append(text[0])
                        data.append(text[1])
                        insert_data.append(data)

                    insert_df = pd.DataFrame(data=insert_data)
                    insert_df.to_csv("./data/coverletter_df.csv", mode="a", header=False, index=False, encoding="utf-8-sig")

                except:
                    pass
        print(str(page) + " 페이지 완료!")
        page += 1
    
    print(section_name + " 완료!")